In [4]:
import pandas as pd
import numpy as np
import os

# data 폴더 경로
data_path = '../data/train'

# train_ratings.csv 파일 경로
train_path = os.path.join(data_path, 'train_ratings.csv')

# train data read
train_data = pd.read_csv(train_path)

# user X item 행렬 및 유저당 아이템 카운트
user_items = train_data.groupby('user')['item'].apply(list).reset_index(name='items')
user_items['item_count'] = user_items['items'].apply(len)

user_items



,user,items,item_count
0,11,"[4643, 170, 531, 616, 2140, 2722, 2313, 2688, ...",376
1,14,"[8961, 1396, 471, 2105, 1042, 1947, 1269, 2394...",180
2,18,"[1952, 1283, 3507, 4280, 51084, 593, 318, 356,...",77
3,25,"[261, 22, 2161, 3255, 372, 1093, 428, 175, 214...",91
4,31,"[260, 1196, 1210, 7153, 4993, 5952, 1270, 5855...",154
...,...,...,...
31355,138473,"[524, 3354, 1025, 6565, 69757, 2085, 32, 55282...",63
31356,138475,"[1639, 1673, 1148, 246, 2019, 1267, 1172, 1235...",124
31357,138486,"[2694, 1994, 2723, 441, 2288, 637, 2013, 2423,...",137
31358,138492,"[2115, 908, 58, 2700, 2599, 1500, 1358, 1288, ...",68


In [8]:
# 앙상블할 기존의 제출 데이터

# bert(sequence) predictions
output_bert = pd.read_csv('./output_bert4rec_42.csv')

# ease(static) predictions
output_ease = pd.read_csv('./output_ease.csv')

In [9]:
# static 모델의 예측값중 몇개를 사용할 것인지에 대한 함수
def calculate_static_limit(count):
    k = 10
    val = count / 100
    if val >= k:
        return k - 1
    # elif val < 1:
    #     return k // 5
    else:
       return int((val / (val + 1)) * k)

# 각 유저마다(row) 앙상블 적용함수
def combine_ensemble(row, k=10):
    limit = int(row['limit_k'])
    
    # 1) BERT에서 가져올 것 (limit 개수만큼)
    bert_selected = row['top_k_bert'][:k - limit]
    
    # 2) EASE에서 가져올 것 (나머지 채우기)
    # 단순히 [:k-limit]만 가져오면, 중복 제거 후 개수가 모자랄 수 있음.
    # 그래서 BERT 전체 리스트를 순회하며 채워넣음.
    final_list = list(bert_selected) # 복사
    
    ease_candidates = row['top_k_ease']
    
    for item in ease_candidates:
        if len(final_list) >= k: # 10개가 찼으면 중단
            break
        if item not in final_list:
            final_list.append(item)

            
    # # 혹시라도 BERT로도 10개가 안 채워졌다면? (BERT 리스트가 짧은 경우 등)
    # # EASE의 나머지 부분에서 추가로 채움
    # if len(final_list) < k:
    #     for item in row['top_k_ease'][limit:]:
    #         if len(final_list) >= k:
    #             break
    #         if item not in final_list:
    #             final_list.append(item)
                
    return final_list

# 앙상블 적용후 제출 df 생성
def ensemble():
    k = 10
    top_k_output_bert = output_bert.groupby('user')['item'].apply(list).reset_index(name='top_k_bert')[['user','top_k_bert']]
    top_k_output_ease = output_ease.groupby('user')['item'].apply(list).reset_index(name='top_k_ease')[['user','top_k_ease']]
    
    merged_df = top_k_output_ease.merge(user_items, on='user', how='left')
    merged_df = top_k_output_bert.merge(merged_df, on='user', how='left')
    merged_df['limit_k'] = merged_df['item_count'].apply(calculate_static_limit)
    
    merged_df['ensemble_list'] = merged_df.apply(lambda row: combine_ensemble(row, k=10), axis=1)
    # print(merged_df)
    return merged_df[['user', 'ensemble_list']]

In [10]:
# 앙상블 실행
ensemble_data = ensemble()


## submission 제출파일 생성


# 1. 리스트를 행으로 펼치기 (explode 사용)
# ensemble_list 컬럼의 [item1, item2, ...] 리스트가 각각의 행으로 분리됩니다.
submission_df = ensemble_data.explode('ensemble_list')

# 2. 컬럼 이름 변경 (ensemble_list -> item)
submission_df = submission_df.rename(columns={'ensemble_list': 'item'})

# 3. 필요한 컬럼만 선택 ('user', 'item' 순서)
final_submission = submission_df[['user', 'item']]

# 4. CSV 파일로 저장
# index=False를 해야 불필요한 인덱스 번호가 저장되지 않습니다.
final_submission.to_csv('submission_4.csv', index=False)

# 결과 확인
print(final_submission.head())

   user   item
0    11  55820
0    11  30707
0    11   8950
0    11   4370
0    11   4886
